In [48]:
rm(list =  ls())
library(dplyr)
data(iris)
head(iris)

# 클러스터링 함수!
kmeans_ <- function(df, cens) {
    label <- c()
    for(i in 1:nrow(df)) {
        min <- 1000000
        min_index <- 0 
        for(cen in 1:nrow(cens)){
    #         print(centroid[cen,])
            temp <- dist(rbind(cens[cen,],df[i,]))
            if(min > temp){
                min <- temp
                min_index <- cen
            }
        }
        label <- c(label, min_index)
    }
    df$label <- label
    return (df)
}

# 군집별 데이터의 중심점 구하기
get_mean <- function(df){
    return (df %>% group_by(label) %>% summarize(Sepal.Length = mean(Sepal.Length), Sepal.Width = mean(Sepal.Width), Petal.Length = mean(Petal.Length), Petal.Width = mean(Petal.Width)))
}

#군집별 중심점과 군집별 각 행의 거리 구하기
get_dist_by_mean <- function(df,mean_){
    dist_ <- c()
    for(i in 1:nrow(df)){
        print(mean_[df[i,]$label,])
        temp <- dist(rbind(mean_[df[i,]$label,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')], df[i,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')]))
        dist_ <- c(dist_, temp)
     }
    return (dist_)
}

# label 별 새로운 중심점 구하기
get_centroid <- function(df){
    a <- df[df$label == 1,]
    b <- df[df$label == 2,]
    c <- df[df$label == 3,]
    
    centroid1 <- a[a$dist == min(a$dist),]
    centroid2 <- b[b$dist == min(b$dist),]
    centroid3 <- c[c$dist == min(c$dist),]
    
    return (rbind(rbind(centroid1,centroid2),centroid3))
}

#중심정 구하는 과정 함수
renew_centroid <- function(df,cens){
    label_result = kmeans_(df, cens)
    mean_table = get_mean(label_result)
    temp_ = get_dist_by_mean(label_result, mean_table)
    label_result$dist = temp_
    new_centroid = get_centroid(label_result)
    return (new_centroid)
}

#### 데이터 셋팅
label <- iris$Species
data_iris <- iris[c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')]
data_iris$index <- (1:150)

df <- data_iris

# centroid
centroid = data_iris[sample(nrow(data_iris), 3), ]

master <- TRUE
centroid2 <- renew_centroid(df, centroid)
# centroid


#### 조건에 만족하는 중심점을 구할때까지 무한루프 반복
while(master == TRUE) {
    a <- dist(rbind(centroid[1,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')], centroid2[1,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')]))
    b <- dist(rbind(centroid[2,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')], centroid2[2,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')]))
    c <- dist(rbind(centroid[3,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')], centroid2[3,c('Sepal.Length','Sepal.Width', 'Petal.Length', 'Petal.Width')])) 
    dist_cens <- mean(a,b,c)
    if(dist_cens > 0.01){ #0.01이하 정도의 차이로 centroid가 변하면 중심과 유사하다는 판결을 내리고 while문 종료
        print("go")
        print(centroid)
        centroid <- centroid2
        print(centroid)
        centroid2 <- renew_centroid(df, centroid[ , -which(names(centroid) %in% c("label","dist"))])
    }else{
        master <- FALSE
        print("complete")
    }
}

### 최종 중심점
print(centroid2)

#### 최종 중심정으로 구한 라벨링
plot_data <- kmeans_(data_iris, centroid2[ , -which(names(centroid) %in% c("label","dist"))])

plot_data[ , -which(names(centroid) %in% c("index"))]